## Importing Libraries

In [4]:
import pandas as pd
import pdfplumber

## Converting the PDF file into a DataFrame

In [5]:
def mypdf(My_pdf,My_number):
    pdf_path = My_pdf # ENTER YOUR PDF PATH HERE
    My_num= My_number

    

    with pdfplumber.open(pdf_path) as pdf:
        first_page = pdf.pages[int(My_num)]
        pdf_text  = first_page.extract_text()
    #print(pdf_text[30:])
    #my_str_2 = pdf_text[31:]
    #my_list_2 = re.split(r',|-|•|\n', my_str_2)
    #print(pdf_text)
    def text_split(text, depth):
        depth_list = ['\n']
        if isinstance(text, str):
            text = text.strip('. ')
            text = text.split(depth_list[depth])
        if depth >= 0:
            depth -= 1
            for ix, item in enumerate(text):
                    item = item.strip('. ')
                    text[ix] = text_split(item, depth)
        return text
    tobe_df= text_split(pdf_text[31:], -1)
    i=0
    new_list=[]
    while i<len(tobe_df):
        new_list.append(tobe_df[i:i+3])
        i+=3
    
    return(pd.DataFrame(new_list))

In [7]:
df = mypdf("Attached A.pdf",0)
for i in range(32):
    df2 = mypdf("Attached A.pdf",i)
    df= pd.concat([df,df2],axis=0)

## Spliting and cleaning the DataFrame into the Desired format

In [8]:
df= df.reset_index()

In [9]:
df3= df.drop('index',axis=1)

In [10]:
df3

,0,1,2
0,"David Abel, CMCA - The Dartmouth Group, An Ass...","100 Alden St • Provincetown, MA 02657",508-487-4771 • dabel@thedartmouthgroup.com
1,"Christina Abreu - Brigs, LLC, AAMC","163 Main St, Ste 201 • Salem, NH 03079",603-403-5294 • cabreu@brigsllc.com
2,Erin Adams - Checkmate Sales and Management,"12 Sterling Way • Hudson, NH 03051",603-809-7016 • erin@checkmate247.com
3,Owen Ahearn - Ahearn Realty Management,"7010 Oak Point Dr • Middleboro, MA 02346",617-851-3139 • owen@ahearnrealty.com
4,"Mary Alderman, CMCA, AMS - Brigs, LLC, AAMC","22 Mill St, • Danvers, MA 01923",617-236-0090 • malderman@brigsllc.com
...,...,...,...
520,Chris Yule - Bristol South Inc.,"814 Broadway, PO Box 77 • Raynham, MA 02767",508-823-2300 • chris@bristolsouth.net
521,Russ Yule - Bristol South Inc.,"PO Box 77 • Raynham, MA 02767",508-823-2300 • offi ce@bristolsouth.net
522,"Ronda Ziner, CPM - E.P. Management Corp.","7 Tozer Rd • Beverly, MA 01915",978-232-1126 • ronda@epmanagement.com
523,Stasia Zomkowski,"1 Huntington Ave • Boston, MA 02116",617-309-6433 • stasia_1@hotmail.com


In [11]:
df3['Contacts']= df3[0].str.split(',|-',expand=True)[0]

In [12]:
df3['First Name']= df3['Contacts'].str.split(' ',expand= True)[0]
df3['Last Name']= df3['Contacts'].str.split(' ',expand= True)[1]

In [13]:
df3['DIST']= df3[0].str.split(',',expand=True)[1].str.split('-',expand= True)[0]

In [14]:
df3['Company']= df3[0].str.split('-',expand=True)[1]

In [15]:
df['Street']= df3[1].str.split('•',expand=True)[0]

In [16]:
df3['City']= df3[1].str.split('•',expand=True)[1].str.split(',| ',expand=True)[1]
df3['State']= df3[1].str.split('•',expand=True)[1].str.split(',| ',expand=True)[3]
df3['Zip']= df3[1].str.split('•',expand=True)[1].str.split(',| ',expand=True)[4]

In [17]:
df3['Phone'] = df3[2].str.split('•',expand= True)[0]
df3['Email'] = df3[2].str.split('•',expand= True)[1]

In [18]:
Final_df = df3[['Contacts', 'First Name',
        'Last Name',       'DIST',    'Company',       'City',      'State',
              'Zip',      'Phone',      'Email']]

## Final look of the Dataframe and exporting it as the desired Excel File

In [19]:
Final_df

,Contacts,First Name,Last Name,DIST,Company,City,State,Zip,Phone,Email
0,David Abel,David,Abel,CMCA,"The Dartmouth Group, An Associa Company AAMC",Provincetown,MA,02657,508-487-4771,dabel@thedartmouthgroup.com
1,Christina Abreu,Christina,Abreu,LLC,"Brigs, LLC, AAMC",Salem,NH,03079,603-403-5294,cabreu@brigsllc.com
2,Erin Adams,Erin,Adams,None,Checkmate Sales and Management,Hudson,NH,03051,603-809-7016,erin@checkmate247.com
3,Owen Ahearn,Owen,Ahearn,None,Ahearn Realty Management,Middleboro,MA,02346,617-851-3139,owen@ahearnrealty.com
4,Mary Alderman,Mary,Alderman,CMCA,"Brigs, LLC, AAMC",Danvers,MA,01923,617-236-0090,malderman@brigsllc.com
...,...,...,...,...,...,...,...,...,...,...
520,Chris Yule,Chris,Yule,None,Bristol South Inc.,Raynham,MA,02767,508-823-2300,chris@bristolsouth.net
521,Russ Yule,Russ,Yule,None,Bristol South Inc.,Raynham,MA,02767,508-823-2300,offi ce@bristolsouth.net
522,Ronda Ziner,Ronda,Ziner,CPM,E.P. Management Corp.,Beverly,MA,01915,978-232-1126,ronda@epmanagement.com
523,Stasia Zomkowski,Stasia,Zomkowski,None,None,Boston,MA,02116,617-309-6433,stasia_1@hotmail.com


In [ ]:
Final_df.to_excel("Final Attach.xlsx")